In [1]:
# For each labels one slope agle, one apsect angle is calculated

In [2]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import rasterio
from rasterio.features import shapes
from rasterio.mask import mask
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio as rio
from shapely.geometry import Polygon
from mpl_toolkits.basemap import Basemap  
import matplotlib.pyplot as plt
from shapely.ops import unary_union
import os
import numpy as np
import rasterio as rio
from shapely.geometry import box

In [3]:
def calculate_slope(row, slope_tif):
    # Read slope raster
    with rasterio.open(slope_tif) as src:
        # Mask the raster using the geometry of the row
        slope_data, _ = mask(src, [row.geometry], crop=True)
    
    # Apply condition to filter out extreme slope values and replace them with NaN
    slope_data = np.where((slope_data > 90) | (slope_data < 0), np.nan, slope_data)
    
    # Extract slope values
    slope_values = slope_data.flatten()
    slope_values = slope_values[~np.isnan(slope_values)]  # Remove NaN values
    
    # Calculate mean and median slope angles
    mean_slope_angle = np.mean(slope_values)
    
    return mean_slope_angle


def calculate_aspect(row, aspect_tif):
    # Read aspect raster
    with rasterio.open(aspect_tif) as src:
        # Mask the raster using the geometry of the row
        aspect_data, _ = mask(src, [row.geometry], crop=True)
    
    # Apply condition to filter out invalid aspect values and replace them with NaN
    aspect_data = np.where((aspect_data > 360) | (aspect_data < 0), np.nan, aspect_data)
    
    # Extract aspect values
    aspect_values = aspect_data.flatten()
    aspect_values = aspect_values[~np.isnan(aspect_values)]  # Remove NaN values
    
    # Calculate mean and median aspect angles (in degrees)
    mean_aspect_angle = np.z(aspect_values)
    
    return mean_aspect_angle


def calculate_slope_for_objects(objects, slope_tif):
    # Iterate over each row (geometry) in the GeoPandas DataFrame and calculate aspect statistics
    for index, row in objects.iterrows():
        slope_value = calculate_slope(row, slope_tif)
        objects.loc[index, 'mean_slope_angle'] = slope_value
    return objects

def calculate_aspect_for_objects(objects, aspect_tif):
    # Iterate over each row (geometry) in the GeoPandas DataFrame and calculate aspect statistics
    for index, row in objects.iterrows():
        aspect_value = calculate_aspect(row, aspect_tif)
        objects.loc[index, 'mean_aspect_angle'] = aspect_value  # Adding mean aspect angle as a new column
    return objects

In [4]:
regions = ['herschel', 'gydan', 'peel', 'kolguev']  # Add more regions if needed
types = ['DEM', 'OP', 'OT']

for region in regions:
    print(region)
    for type in types:
        print(type)
        input_filepath = f'Results\\all\\{region}_{type}_labels_results.geojson'
        output_filepath = f'Results\\all\\{region}_{type}_labels_results.geojson'
        terrain_directory = f'terrain_attributes\\{region}_'

        # Read GeoJSON file
        objects = gpd.read_file(input_filepath)

        # Calculate mean slope angle for each object
        objects = calculate_slope_for_objects(objects, terrain_directory + 'slopes.tif')

        objects = calculate_aspect_for_objects(objects, terrain_directory + 'aspects.tif')

        # Save the modified GeoJSON file
        objects.to_file(output_filepath, driver='GeoJSON')
        objects.to_file(f'Results\\{region}\\{region}_{type}_labels_results.geojson', driver='GeoJSON')

herschel
DEM


AttributeError: module 'numpy' has no attribute 'z'